In [1]:
import pandas as pd
import folium
import numpy as np
import glob
from folium.plugins import MarkerCluster

In [2]:
stations = pd.read_csv('inventario.csv', header=0, delimiter=',')

In [3]:
stations = stations.replace('#REF!', np.NaN)
stations = stations.replace('#N/D', np.NaN)


In [4]:
stations.iloc[[88,436]] #88 had a #REF! and 436 had a #N/D! problem

,ID HISTÓRICO (itep),Cód. IBGE/Apac,MUNICÍPIO,ESTAÇÃO,LAT,LON,ALT,LOCAL,TIPO PCD,TIPO,TIPO COLETA,OPERAÇÃO,BACIA HIDROGRÁFICA,MICRORREGIÃO,HOMOGENIA,MESORREGIÃO,DATA INSTALAÇÃO,DATA DESINSTALAÇÃO,GOOGLE
88,2196,260140901H,BARREIROS,Rio Caruma [H],NaN,NaN,0.0,BARREIROS,Hidrometeorologica,Fluviometrica,Automática,CEMADEN,0,Mata Sul,Mata Pernambucana,Mata Pernambucana,1/1/1970,4/30/1993,NaN
436,2143,260730701C,IPUBI,IPUBI [C],NaN,NaN,0.0,IPUBI,Meteorologica,Pluviometrica,Automática,CEMADEN,0,Sertão do Araripe,Sertão Pernambucano,Sertão Pernambucano,1/1/1970,10/31/1999,NaN


In [5]:
stations['LAT'] = pd.to_numeric(stations['LAT'])
stations['LON'] = stations['LON'].astype('float')

In [6]:
stations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1067 entries, 0 to 1066
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID HISTÓRICO (itep)  1047 non-null   object 
 1   Cód. IBGE/Apac       1067 non-null   object 
 2   MUNICÍPIO            1067 non-null   object 
 3   ESTAÇÃO              1060 non-null   object 
 4   LAT                  1063 non-null   float64
 5   LON                  1063 non-null   float64
 6   ALT                  1047 non-null   float64
 7   LOCAL                1067 non-null   object 
 8   TIPO PCD             1067 non-null   object 
 9   TIPO                 1066 non-null   object 
 10  TIPO COLETA          1066 non-null   object 
 11  OPERAÇÃO             1066 non-null   object 
 12  BACIA HIDROGRÁFICA   1046 non-null   object 
 13  MICRORREGIÃO         1046 non-null   object 
 14  HOMOGENIA            1066 non-null   object 
 15  MESORREGIÃO          1066 non-null   o

In [7]:
stations_without_nan = stations[stations['LAT'].notna()] #only stations without nan values for LAT and LON

In [98]:
table = """
<!DOCTYPE html>
<html>
<head>
<style>
table {{
    width:100%;
    border-radius:10px;
}}
table, th, td {{
    border-collapse: collapse;
    
}}
th, td {{
    padding: 5px;
    text-align: left;
}}
#t01 tr:nth-child(odd) {{
    background-color:rgba(9, 0, 181, 0.2);
}}
#t01 tr:nth-child(even) {{
   background-color:rgba(186, 190, 204, 0.4);
}}
#t01 th {{
  background-color: #040054;
  color: white;
}}
table tr:hover td {{
  background-color: #ddd;
}}


<! -- Rounding borders of table -- >
table th:first-child {{
 border-radius: 5px 0 0 0;
}}
table th:last-child {{
  border-radius: 0 5px 0 0;
}}
table tr:last-child td:first-child {{
  border-radius: 0 0 0 5px;
}}
table tr:last-child td:last-child {{
  border-radius: 0 0 5px 0;
}}


</style>
</head>
<body>

<table id="t01">
  <tr>
    <th>{}</th>
    <th>{}</th>
  </tr>
  <tr>
    <td>{}</td>
    <td>{}</td>
  </tr>
  <tr>
    <td>{}</td>
    <td>{}</td>
  </tr>
  <tr>
    <td>{}</td>
    <td>{}</td>
  </tr>
  <tr>
    <td>{}</td>
    <td>{}</td>
  </tr>
  <tr>
    <td>{}</td>
    <td>{}</td>
  </tr>

</table>
</body>
</html>
""".format

In [110]:
#Base map
map = folium.Map(location=[-6.5, -37], zoom_start = 7, min_zoom=6, max_zoom=9)

#options of tiles
folium.TileLayer('openstreetmap').add_to(map)
folium.TileLayer('Stamen Terrain').add_to(map)
folium.TileLayer('cartodbdark_matter').add_to(map)

marker_cluster = MarkerCluster(name='Estações').add_to(map)

#Plot markers of the stations
for _, local in stations_without_nan.iterrows():
  iframe = folium.IFrame(table(
                        "ESTAÇÃO " , str(local['ESTAÇÃO']),
                        "ID " , str(local['Cód. IBGE/Apac']),    
                        "LATITUDE " , str(local['LAT']),
                        "LONGITUDE " , str(local['LON']),                      
                        "TIPO PCD " , str(local['TIPO PCD']), 
                        "TIPO COLETA " , str(local['TIPO COLETA'])
                               )
                          )

  popup = folium.Popup(iframe,
                     min_width=500,
                     max_width=750)

  pcd_icon = folium.features.CustomIcon('pcd_icon.jpg',
                                      icon_size=(50, 50))
                                      
  folium.Marker(location=[local['LAT'], local['LON']], 
                popup=popup, 
                icon=pcd_icon,
                tooltip='Clique para ver os detalhes').add_to(marker_cluster)

#Add style to regions
style_function = lambda x: {'fillColor': 'black', 'opacity':0.2, 'color':'black', 'weight':5, 'dashArray':'5, 5', 'fillOpacity':0.2}   #https://leafletjs.com/SlavaUkraini/reference.html#path-option

#add all regioes to map
files = glob.glob('./mesoregioes_geojson/*.geojson')

for f in files:
  region = f.replace("\\", "/") #correction. Glob returns \ instead of /
  region_name = region.split('/')[-1].split('.')[0] #https://stackoverflow.com/questions/7336096/python-glob-without-the-whole-path-only-the-filename#answer-68415441
  layer = folium.GeoJson(region, control='true',   style_function=style_function, name=region_name)
  #folium.GeoJsonTooltip(fields=region_name['nome']).add_to(map)
  folium.GeoJsonTooltip(fields=["nome"]).add_to(layer)
  layer.add_to(map)
  
folium.LayerControl().add_to(map)
map

AssertionError: The field nome is not available in the data. Choose from: ('OBJECTID', 'OBJECTID_1', 'REGDESENV', 'Shape_Area', 'Shape_Le_1', 'Shape_Leng', 'RBSTYLE_0', 'RBSTYLE_1', 'RBSTYLE_2').